In [1]:
import pysam
import numpy as np
import cPickle as pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from collections import Counter
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.INFO)
from operator import itemgetter
import pandas as pd

In [2]:
import os

In [3]:
def fastq_to_names(in_file):
    with open(in_file) as fq:
        content = fq.readlines()
    
    names = []
    for line in content:
        if 'L183:338:CAGAAANXX' in line:
            names.append(line.rstrip()[1:])
    
    return names

In [4]:
names = fastq_to_names('/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.fastq')

In [8]:
def umi_to_umis(in_file):
    with open(in_file) as fu:
        content = fu.readlines()
    
    umis = []
    for line in content:
        umis.append(line.rstrip())
    
    return umis

In [9]:
umis = umi_to_umis('/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.umi')

In [10]:
def umi_to_name_map(fq_file, umi_file):
    
    names = fastq_to_names(fq_file)
    umis = umi_to_umis(umi_file)
    un_dict = dict(zip(names, umis))
    
    return un_dict

In [11]:
un = umi_to_name_map('/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.fastq','/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.umi')

In [13]:
def xm_tag_reads(bam_file, fq_file, umi_file):
    
    umi_name_map = umi_to_name_map(fq_file, umi_file)
    
    st = pysam.AlignmentFile(bam_file, "rb")
    
    out_file = "%s.tagged.bam" % os.path.splitext(bam_file)[0]
    tagged_reads = pysam.AlignmentFile('tmp.bam', 'wb', template=st)
    
    st.reset()
    reads = st.fetch(until_eof=True)
    for r in reads:
        if r.has_tag('RG'):
            r.set_tag('RG', None)
        xm = umi_name_map[r.query_name]
        r.tags += [('XM', xm)]
        tagged_reads.write(r)

    tagged_reads.close()
    pysam.sort("-o", out_file, 'tmp.bam')
    pysam.index(out_file)

In [16]:
ff = '/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.fastq'
uf = '/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.umi'
bf = '/data/parastou/Star-Lab/test/NStar25.Aligned.out.bam'

In [17]:
xm_tag_reads(bf, ff, uf)